In [4]:
import pandas as pd
import numpy as np

In [5]:
df_ex = pd.read_csv("./resources/data/exercise.csv", header=0)
df_workout = pd.read_csv("./resources/data/workout.csv", header=0)
df_user = pd.read_csv("./resources/data/user.csv", header=0)
df_derived_ex = pd.read_csv("./resources/data/exercise_derived.csv", header=0)

In [84]:
# print(df_ex.head(1))
# print(df_ex.dtypes)
# print(df_workout.head(1))
# print(df_workout.dtypes)
# print(df_user.head(1))
# print(df_user.dtypes)
# print(df_derived_ex.head(1))
# print(df_derived_ex.dtypes)

In [85]:
the_columns_exd = df_derived_ex.columns
the_columns_ex = df_ex.columns

In [98]:
print(the_columns_exd)
print(the_columns_ex)

Index(['id', 'name', 'exercise_importance', 'exercise_cost',
       'exercise_recurrance_factor_per_cycle', 'exercise_rpe',
       'exercise_overall_fun_factor', 'exercise_exhaustion_factor',
       'missed_in_cycle_count'],
      dtype='object')
Index(['id', 'name', 'exercise_type', 'muscles_targeted', 'calories_10_min',
       'fun_factor', 'is_compound', 'how_critical'],
      dtype='object')


In [6]:
df_derived_ex['exercise_importance'] = 1

In [146]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2,burpee,1,NaN,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,1,NaN,NaN,NaN,NaN,NaN,NaN
3,4,squat,1,NaN,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,1,NaN,NaN,NaN,NaN,NaN,NaN
5,6,push_up,1,NaN,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,1,NaN,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,1,NaN,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1,NaN,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
# Validation the merged columns will be 15
k = set(df_ex.columns)
c = set(df_derived_ex.columns)
for i in c:
    k.add(i)
print(len(k))
k

15


{'calories_10_min',
 'exercise_cost',
 'exercise_exhaustion_factor',
 'exercise_importance',
 'exercise_overall_fun_factor',
 'exercise_recurrance_factor_per_cycle',
 'exercise_rpe',
 'exercise_type',
 'fun_factor',
 'how_critical',
 'id',
 'is_compound',
 'missed_in_cycle_count',
 'muscles_targeted',
 'name'}

In [99]:
# Join the datasets
merged_df = pd.merge(df_ex, df_derived_ex, on=['id', 'name'])

In [122]:
merged_df.columns

Index(['id', 'name', 'exercise_type', 'muscles_targeted', 'calories_10_min',
       'fun_factor', 'is_compound', 'how_critical', 'exercise_importance',
       'exercise_cost', 'exercise_recurrance_factor_per_cycle', 'exercise_rpe',
       'exercise_overall_fun_factor', 'exercise_exhaustion_factor',
       'missed_in_cycle_count'],
      dtype='object')

In [102]:
df_user.head(10)

,id,name,gender,fitness_level,focused_muscle_group,age,goal,weight
0,1,Mike,0,1,bicep tricep back leg,20,strength,90
1,2,Sam,0,2,leg,45,bulking,50
2,3,Monica,1,2,tricep leg,30,strength,60
3,4,John,0,1,core leg back,18,wt_loss,100
4,5,Max,0,0,tricep chest shoulder,60,strength,50
5,6,Rachel,1,0,shoulder chest tricep,25,strength,55
6,7,Dilpreet,0,1,shoulder core leg,70,bulking,70
7,8,Diljeet,0,2,leg core bicep back tricep,35,wt_loss,120
8,9,Reshma,1,2,chest shoulder tricep back,47,strength,60
9,10,Rahul,0,1,back,22,wt_loss,130


In [118]:
# If bicep is in user focus column
"bicep" in df_user[df_user['name'] == 'Mike']['focused_muscle_group'].item().split()

True

In [170]:
def update_weights(row):
    if "bicep" in df_user[df_user['name'] == 'Mike']['focused_muscle_group'].item().split():
        # Update all rows who have bicep in muscle_targeted
        if 'bicep' in row['muscles_targeted'].split():
            id = row['id']
            old_val = df_derived_ex[df_derived_ex['id'] == id]['exercise_importance'].item()
            df_derived_ex.loc[df_derived_ex['id'] == id, 'exercise_importance'] = old_val + 0.37

In [14]:
def update_weights_muscle_group_matching(df_user, df_derived_ex, row, user_name, muscle_group, weightage):
    if muscle_group in df_user[df_user['name'] == user_name]['focused_muscle_group'].item().split():
        # Update all rows who have bicep in muscle_targeted
        if muscle_group in row['muscles_targeted'].split():
            id = row['id']
            old_val = df_derived_ex[df_derived_ex['id'] == id]['exercise_importance'].item()
            df_derived_ex.loc[df_derived_ex['id'] == id, 'exercise_importance'] = old_val + weightage

In [15]:
# df_user, df_derived_ex, row, user_name, muscle_group, weightage
import KnowledgeBase
df_ex.apply(lambda row: update_weights_muscle_group_matching(df_user, df_derived_ex, row, "Mike", "bicep", 0.37), axis = 1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
dtype: object

In [18]:
mapping = {"bicep":0.25, "chest": 0.55, "core": 0.20, "leg": 0.45, "back": 0.55, "shoulder": 0.45, "tricep": 0.37}

In [20]:
for key, value in mapping.items():
    print(key)
    print(value)

ValueError: too many values to unpack (expected 2)

In [171]:

df_ex.apply(lambda row: update_weights(row), axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
dtype: object

In [17]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,bicep_curl,1.37,NaN,NaN,NaN,NaN,NaN,NaN
1,2,burpee,1.00,NaN,NaN,NaN,NaN,NaN,NaN
2,3,deadlift,1.00,NaN,NaN,NaN,NaN,NaN,NaN
3,4,squat,1.00,NaN,NaN,NaN,NaN,NaN,NaN
4,5,bench_press,1.00,NaN,NaN,NaN,NaN,NaN,NaN
5,6,push_up,1.00,NaN,NaN,NaN,NaN,NaN,NaN
6,7,pull_up,1.37,NaN,NaN,NaN,NaN,NaN,NaN
7,8,lat_pulldown,1.37,NaN,NaN,NaN,NaN,NaN,NaN
8,9,leg_press,1.00,NaN,NaN,NaN,NaN,NaN,NaN
9,10,leg_extension,1.00,NaN,NaN,NaN,NaN,NaN,NaN


,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,Bicep Curl,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Burpee,1,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Deadlift,1,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Squat,1,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Bench Press,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_df = pd.merge(df1, df2, on=['col1', 'col2'])

In [45]:
df_derived_ex.head(10)

,id,name,exercise_importance,exercise_cost,exercise_recurrance_factor_per_cycle,exercise_rpe,exercise_overall_fun_factor,exercise_exhaustion_factor,missed_in_cycle_count
0,1,Bicep Curl,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Burpee,1,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Deadlift,1,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Squat,1,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Bench Press,1,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Push-up,1,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Pull-up,1,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Lat Pulldown,1,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Leg Press,1,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Leg Extension,1,NaN,NaN,NaN,NaN,NaN,NaN
